## 使用apikey的事例

In [1]:

import os
import logging
import re

from binance_sdk_spot.spot import Spot, ConfigurationRestAPI, SPOT_REST_API_PROD_URL

# Configure logging
logging.basicConfig(level=logging.INFO)

# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key="66aLkztQkQFuyUQuM83oqYx9ENuZH1SFFbuTSP3ClcGYa9Vqwy5cmQan9QfK7Gzd",
    api_secret="V4lTk3PUCa7jwGAxPybBFzE7fN2Ob09FZR9BbchLOiGtssRnH9cnkfL2O8C7aoQW",
    base_path= "https://api.binance.com",
)

# Initialize Spot client
client = Spot(config_rest_api=configuration_rest_api)


def ticker24hr():
    try:
        response = client.rest_api.ticker24hr()

        rate_limits = response.rate_limits
        # logging.info(f"ticker24hr() rate limits: {rate_limits}")

        data = response.data()
        for t in data:
            if t[0] == "actual_instance":    
                return t[1]
    # logging.info(f"ticker24hr() response: {data}")
    except Exception as e:
        logging.error(f"ticker24hr() error: {e}")
        return []

def in_exchange_trading_symbols():
    try:
        response = client.rest_api.exchange_info()

        rate_limits = response.rate_limits
        # logging.info(f"exchange_info() rate limits: {rate_limits}")

        data = response.data()
        pattern =  r"usdt$"
        usdt_symbols= [
            t.symbol for t in data.symbols
            if re.search(pattern, t.symbol, flags=re.IGNORECASE) and t.status=='TRADING'
        ]
        return usdt_symbols
        logging.info(f"exchange_info() response: {data}")
    except Exception as e:
        logging.error(f"exchange_info() error: {e}")

def get_top3_gainers():
    """筛选当日涨幅前三的币种"""
    tickers = ticker24hr()

    if not tickers:
        return []

    in_trading_symbols = in_exchange_trading_symbols()

    if not in_trading_symbols:
        return []


    pattern =  r"usdt$"
    usdt_tickers= [
        t for t in tickers
        if re.search(pattern, t.symbol, flags=re.IGNORECASE)
    ]

    in_trading_tickers = [
        t for t in usdt_tickers if t.symbol in in_trading_symbols
    ]
    
    valid_tickers = [
        t for t in  in_trading_tickers
        if t.price_change_percent and not t.symbol.endswith(("UP", "DOWN", "USDTM"))  # 排除杠杆/合约交易对
    ]
    
    sorted_tickers = sorted(
        valid_tickers,
        key=lambda x: float(x.price_change_percent),
        reverse=True
    )
    
    return sorted_tickers[:3]

if __name__ == "__main__":
    tickers = get_top3_gainers()
    print("Top 3 Gainers:")
    for t in tickers:
        print(f"Symbol: {t.symbol}, Change: {t.price_change_percent}%, Open: {t.open_price}, Close: {t.last_price}")

Top 3 Gainers:
Symbol: A2ZUSDT, Change: 34.814%, Open: 0.00139600, Close: 0.00188200
Symbol: PEPEUSDT, Change: 26.355%, Open: 0.00000406, Close: 0.00000513
Symbol: NEIROUSDT, Change: 22.358%, Open: 0.00010296, Close: 0.00012598


## 使用免费接口的事例

In [127]:
import requests
import re
import logging

def in_exchange_trading_symbols():
    url = "https://data-api.binance.vision/api/v3/exchangeInfo"
    try:
        response = requests.get(url, timeout=10)   

        data = response.json()
        pattern =  r"usdt$"
        usdt_symbols= [
            t['symbol'] for t in data['symbols']
            if re.search(pattern, t['symbol'], flags=re.IGNORECASE) and t['status']=='TRADING'
        ]
        return usdt_symbols
        # return data
        logging.info(f"exchange_info() response: {data}")
    except Exception as e:
        logging.error(f"exchange_info() error: {e}")

def get_ticker24hr():
    url = "https://data-api.binance.vision/api/v3/ticker/24hr"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() 
        return response.json()

    except Exception as e:
        print(f"ticker24hr() error: {e}")
        return []

def get_top3_gainers():
    """筛选当日涨幅前三的币种"""
    tickers = get_ticker24hr()
    
    if not tickers:
        return []
    
    in_trading_symbols = in_exchange_trading_symbols()

    if not in_trading_symbols:
        return []

    pattern =  r"usdt$"
    usdt_tickers= [
        t for t in tickers
        if re.search(pattern, t.get('symbol'), flags=re.IGNORECASE)
    ]

    in_trading_tickers = [
        t for t in usdt_tickers if t.get('symbol') in in_trading_symbols
    ]

    valid_tickers = [
        t for t in  in_trading_tickers
        if t.get("priceChangePercent") and not t.get("symbol").endswith(("UP", "DOWN", "USDTM"))  # 排除杠杆/合约交易对
    ]
    
    sorted_tickers = sorted(
        valid_tickers,
        key=lambda x: float(x["priceChangePercent"]),
        reverse=True
    )
    
    return sorted_tickers[:3]
if __name__ == "__main__":

    tickers = get_top3_gainers()
    print("Top 3 Gainers:")
    for t in tickers:
        print(f"Symbol: {t['symbol']}, Change: {t['priceChangePercent']}%, Open: {t['openPrice']}, Close: {t['lastPrice']}")

Top 3 Gainers:
Symbol: STORJUSDT, Change: 36.641%, Open: 0.11490000, Close: 0.15700000
Symbol: ONTUSDT, Change: 25.045%, Open: 0.05510000, Close: 0.06890000
Symbol: NTRNUSDT, Change: 21.862%, Open: 0.02470000, Close: 0.03010000


In [128]:
in_exchange = in_exchange_trading_symbols()

In [130]:
def in_exchange():
    url = "https://data-api.binance.vision/api/v3/exchangeInfo"
    try:
        response = requests.get(url, timeout=10)   

        data = response.json()
        # pattern =  r"usdt$"
        # usdt_symbols= [
        #     t['symbol'] for t in data['symbols']
        #     if re.search(pattern, t['symbol'], flags=re.IGNORECASE) and t['status']=='TRADING'
        # ]
        return data
        # return data
        logging.info(f"exchange_info() response: {data}")
    except Exception as e:
        logging.error(f"exchange_info() error: {e}")

In [131]:
result = in_exchange()

In [134]:
data = result.get('symbols')

In [ ]:
data[0]

In [136]:
def ticker24hr():
    try:
        response = client.rest_api.ticker24hr()

        rate_limits = response.rate_limits
        # logging.info(f"ticker24hr() rate limits: {rate_limits}")

        data = response.data()
        return data
    except Exception as e:
        logging.error(f"ticker24hr() error: {e}")
        return []

In [139]:
rr = ticker24hr()

In [141]:
len(rr)

TypeError: object of type 'Ticker24hrResponse' has no len()